# Demo 1: Fallzahlen, Genesungen, und Todesfällen Vergleichen

## Importing Dependencies

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wgts
from IPython.display import display

## Loading Datasets

In [4]:
# test data read
covid_19_dataset = pd.read_csv('../dataset/time-series-19-covid-combined.csv')
regions_dataset = pd.read_csv('../dataset/Countries by continents.csv')

## Creating Widgets

### Creating Countries & Continents Dropdown Menus

In [23]:
countries_dropdown_menu = wgts.Dropdown(
    options= regions_dataset['Country'].unique(),
    description='Country:',
    disabled=False,
)

continent_dropdown_menu = wgts.Dropdown(
    options= regions_dataset['Continent'].unique(),
    description='Continent:',
    disabled=False,
)

### Creating Regional Category Radio Button

In [24]:
regional_category_radio = wgts.RadioButtons(
    options=['Country', 'Region', 'World'],
    description='Regional Category:',
    disabled=False
)

### Creating Time Category Radio Button

In [27]:
time_category_radio = wgts.RadioButtons(
    options=['Daily', 'Weekly', 'Monthly', 'Complete'],
    description='Time Category:',
    disabled=False
)